En este cuaderno está todo el código necesario para poder importar y consultar los datos del ejercicio, primero todo el proceso de MySQL y luego Cassandra

Carga de datos en MySQL (extracto de los datos, en caso de querer los datos completos ejecutar dataSetGenerator.py y cambiar "extracto_datos.csv" por "datos_educacion.csv", tomará unos 15 minutos cargar todo)

In [ ]:
import mysql.connector
import pandas as pd

# Conectar á base de datos MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="contraseñasegura",
    database="escuela"
)

cursor = conn.cursor()

# Ler o dataset CSV
df = pd.read_csv("extracto_datos.csv",skiprows=1)

# Crear a táboa en MySQL
cursor.execute("""
CREATE TABLE IF NOT EXISTS estudiantes (
    ID_Estudiante INT PRIMARY KEY,
    Nombre VARCHAR(100),
    Grado VARCHAR(50),
    Materia VARCHAR(50),
    Calificacion DECIMAL(3,2),
    Asistencia DECIMAL(5,2),
    Fecha_Registro DATE
)
""")

# Insertar os datos en MySQL
for _, row in df.iterrows():
    cursor.execute("""
    INSERT INTO estudiantes (ID_Estudiante, Nombre, Grado, Materia, Calificacion, Asistencia, Fecha_Registro)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (row['ID_Estudiante'], row['Nombre'], row['Grado'], row['Materia'], row['Calificación'], row['Asistencia (%)'], row['Fecha_Registro']))

conn.commit()
cursor.close()
conn.close()

print("Datos cargados en MySQL!")

Consulta de los datos de MySQL

In [1]:
import mysql.connector
import time

# Conectar a MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="contraseñasegura",
    database="escuela"
)
cursor = conn.cursor()

# Medir o tempo de execución da consulta
start_time = time.time()
cursor.execute("SELECT AVG(Calificacion) FROM estudiantes")
avg_mysql = cursor.fetchone()[0]
end_time = time.time()

print(f"Media en MySQL: {avg_mysql}")
print(f"Tempo de execución en MySQL: {end_time - start_time} segundos")

cursor.close()
conn.close()

AttributeError: module 'ssl' has no attribute 'wrap_socket'

Conexion con la base de datos de Cassandra

In [6]:
from cassandra.cluster import Cluster
import pandas as pd

# Conectar a Cassandra
cluster = Cluster(['localhost'])
session = cluster.connect()


Carga de datos en Cassandra (extracto de los datos, en caso de querer los datos completos ejecutar dataSetGenerator.py y cambiar "extracto_datos.csv" por "datos_educacion.csv", tomará unos 15 minutos cargar todo)

In [ ]:
from cassandra.cluster import Cluster
import pandas as pd

# Conectar a Cassandra
cluster = Cluster(['localhost'])
session = cluster.connect()

# Crear el keyspace y la tabla
session.execute("""
CREATE KEYSPACE IF NOT EXISTS testkeyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
""")
session.set_keyspace('testkeyspace')

session.execute("""
CREATE TABLE IF NOT EXISTS estudiantes (
    ID_Estudiante INT PRIMARY KEY,
    Nombre TEXT,
    Grado TEXT,
    Materia TEXT,
    Calificacion FLOAT,
    Asistencia DECIMAL,
    Fecha_Registro DATE
);
""")

# Leer el dataset CSV
df = pd.read_csv("extracto_datos.csv")

# Insertar los datos
for _, row in df.iterrows():
    session.execute("""
    INSERT INTO estudiantes (ID_Estudiante, Nombre, Grado, Materia, Calificacion, Asistencia, Fecha_Registro) 
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, (row['ID_Estudiante'], row['Nombre'], row['Grado'], row['Materia'], row['Calificación'], row['Asistencia (%)'], row['Fecha_Registro']))

print("Datos cargados en Cassandra!")


Consulta en Cassandra

In [7]:
from cassandra.cluster import Cluster
import time

# Conectar a Cassandra
cluster = Cluster(['localhost'])
session = cluster.connect('testkeyspace')

# Definir la consulta
query = "SELECT AVG(Calificacion) FROM estudiantes"
print(f"Ejecutando consulta en Cassandra: {query}")

# Medir el tiempo de ejecución de la consulta
start_time = time.time()
rows = session.execute(query)

# Usar 'one()' para obtener la primera fila (única fila) y acceder al valor
avg_cassandra = rows.one()[0]  # 'one()' devuelve la primera fila, y [0] es el primer valor de esa fila
end_time = time.time()

session.shutdown()

# Mostrar el resultado
print(f"Promedio de Calificación en Cassandra: {avg_cassandra}")
print(f"Tiempo de ejecución en Cassandra: {end_time - start_time} segundos")

Ejecutando consulta en Cassandra: SELECT AVG(Calificacion) FROM estudiantes
Promedio de Calificación en Cassandra: 5.0042314529418945
Tiempo de ejecución en Cassandra: 4.4554736614227295 segundos


Preguntas propuestas finales:


    ¿Cuál es más rápida?
        MySQL: El tiempo de consulta fue de 0.2120361328125 segundos.

    ¿Son los resultados los esperados?
        La media diverge ligeramente en Cassandra respecto a MySQL.
        Promedio de calificación en Cassandra: 5.0042314529418945.
        Media en MySQL: 4.997203.

    ¿Cómo justificas los resultados?
        Esta pequeña diferencia puede deberse al funcionamiento interno de los decimales y redondeos que aplican ambos lenguajes, especialmente considerando el volumen tan grande de datos con el que se calcula la media.
